Import all the necessary packages

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import random
random.seed(42)

import re


Load the files

In [ ]:
prompts_train=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
data=pd.merge(summaries, prompts_train, on="prompt_id")


Functions to feature engineer the dataset

In [ ]:

def jaccard_similarity(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    intersection = len(words1.intersection(words2))
    union = len(words1) + len(words2) - intersection
    return float(intersection) / float(union)
def cosine_similarity(s1, s2):
    allsentences = [s1 , s2]
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1-cosine)*100,2)
def word_overlap(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    common_words = words1.intersection(words2)
    overlap_count = len(common_words)
    return overlap_count
def semanticsimilarity(text1, text2):
    score=web_model.predict([(text1,text2)])
    return np.array(score)[0]
def sentiment_diff(text1, text2):
    sentiment_score1 = analyzer.polarity_scores(text1)
    sentiment_score2 = analyzer.polarity_scores(text2)
    return abs(sentiment_score1['compound']-sentiment_score2['compound'])
def is_objective(text):
    subjective_words = ["I", "me", "my", "we", "our", "us", "you", "your", "mine", "ours"]
    
    words = text.split()
    
    for word in words:
        if word.lower() in subjective_words:
            return False  
    
    return True  
# lets put it in the data
data['Jaccard_Similarity'] = data.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
data['cosine_similarity'] = data.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
data['word_overlap'] = data.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
#data['sentiment_diff'] = data.apply(lambda row: sentiment_diff(row['prompt_text'], row['text']), axis=1)



below code is obtained from this notebook- https://www.kaggle.com/code/gusthema/commonlit-evaluate-student-summaries-w-tfdf

In [ ]:

def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

# Count the digits in the text.
def count_numbers(text: str) -> int:
    numbers = re.findall(r'\d+', text)
    numbers_count = len(numbers)
    return numbers_count

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].apply(lambda x: len(x.split(' ')))
    dataframe[f'{feature}_length'] = dataframe[feature].apply(lambda x: len(x))
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].apply(lambda x: count_stopwords(x))
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].apply(lambda x: count_punctuation(x))
    dataframe[f'{feature}_number_cnt'] = dataframe[feature].apply(lambda x: count_numbers(x))
    return dataframe

In [ ]:
data.head()


In [ ]:
data=feature_engineer(data)

In [ ]:

data.columns

Separating the data into targer and features

In [ ]:
xtrain=data[[ 'Jaccard_Similarity', 'cosine_similarity', 'word_overlap', 'text_word_cnt', 'text_length', 'text_stopword_cnt', 'text_punct_cnt',
       'text_number_cnt']]
y_wording=data['wording']
y_content=data['content']

In [ ]:
xtrain.shape

In [ ]:
y_wording.shape


# split the data into training and test set

In [ ]:
X_train_wording, X_test_wording, y_train_wording, y_test_wording = train_test_split(xtrain, y_wording, test_size=0.2, random_state=42)
X_train_content, X_test_content, y_train_content, y_test_content = train_test_split(xtrain, y_content, test_size=0.2, random_state=42)

# Fit a random forest model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

base_model= RandomForestRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}


grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search1 = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_wording, y_train_wording)
grid_search1.fit(X_train_content, y_train_content)
best_params = grid_search.best_params_
best_params1=grid_search1.best_params_
model=RandomForestRegressor(**best_params)
model1=RandomForestRegressor(**best_params1)


best_bagging_regressor = BaggingRegressor(model)
best_bagging_regressor1 = BaggingRegressor(model1)

# Fit the best Bagging Regressor to your training data
best_bagging_regressor.fit(X_train_wording, y_train_wording)
best_bagging_regressor1.fit(X_train_content, y_train_content)





In [ ]:
y_preds = best_bagging_regressor.predict(X_test_wording)
mse = mean_squared_error(y_test_wording, y_preds)
mse

In [ ]:
y_preds = best_bagging_regressor1.predict(X_test_content)
mse = mean_squared_error(y_test_content, y_preds)
mse

Loading the test data

In [ ]:
summaries_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
prompts_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
test=pd.merge(summaries_test, prompts_test, on='prompt_id')
test['Jaccard_Similarity'] = test.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
test['cosine_similarity'] = test.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
test['word_overlap'] = test.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
test=feature_engineer(test)


In [ ]:
test.head()

In [ ]:
X_test_data=test[[ 'Jaccard_Similarity', 'cosine_similarity', 'word_overlap', 'text_word_cnt', 'text_length', 'text_stopword_cnt', 'text_punct_cnt',
       'text_number_cnt']]

In [ ]:
y_pred = best_bagging_regressor.predict(X_test_data)
y_pred_content=best_bagging_regressor1.predict(X_test_data)

In [ ]:
submission=pd.DataFrame({'student_id':test.student_id, 'content':y_pred_content, 'wording':y_pred})


In [ ]:
submission.to_csv('submission.csv', index=False)


In [ ]:
submission